# Intro 

This project is in conjunction with the mining company OilyGiant, with the hopes of finding the best location for a new well. With the help of a machine learning model, we will be able to determine which region will be most worthwhile. This will be determined by each region's quality, volume, and estimates. I will begin by preparing the data , then moving on to creating and training the model. The next step will be to calculate various metrics related to profit. With that information, I will be able to select the best region. That is not without the consideration of potential risks and errors, which will be identified with the help of the Bootstrapping technique. With the support from this project, OilyGiant will be able to have the location of a new well while also having the security knowing that it will produce the most for the best amount. 

# Preparing the Data

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
df1 = pd.read_csv('/datasets/geo_data_0.csv') 
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
df1 , df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


(          id        f0        f1        f2     product
 0      txEyH  0.705745 -0.497823  1.221170  105.280062
 1      2acmU  1.334711 -0.340164  4.365080   73.037750
 2      409Wp  1.022732  0.151990  1.419926   85.265647
 3      iJLyR -0.032172  0.139033  2.978566  168.620776
 4      Xdl7t  1.988431  0.155413  4.751769  154.036647
 ...      ...       ...       ...       ...         ...
 99995  DLsed  0.971957  0.370953  6.075346  110.744026
 99996  QKivN  1.392429 -0.382606  1.273912  122.346843
 99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
 99998  7kl59  0.998163 -0.528582  1.583869   74.040764
 99999  1CWhH  1.764754 -0.266417  5.722849  149.633246
 
 [100000 rows x 5 columns],
 None)

In [5]:
df2 , df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


(          id         f0         f1        f2     product
 0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
 1      62mP7  14.272088  -3.475083  0.999183   26.953261
 2      vyE1P   6.263187  -5.948386  5.001160  134.766305
 3      KcrkZ -13.081196 -11.506057  4.999415  137.945408
 4      AHL4O  12.702195  -8.147433  5.004363  134.766305
 ...      ...        ...        ...       ...         ...
 99995  QywKC   9.535637  -6.878139  1.998296   53.906522
 99996  ptvty -10.160631 -12.558096  5.005581  137.945408
 99997  09gWa  -7.378891  -3.084104  4.998651  137.945408
 99998  rqwUm   0.665714  -6.152593  1.000146   30.132364
 99999  relB0  -3.426139  -7.794274 -0.003299    3.179103
 
 [100000 rows x 5 columns],
 None)

In [6]:
df3 , df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


(          id        f0        f1        f2     product
 0      fwXo0 -1.146987  0.963328 -0.828965   27.758673
 1      WJtFt  0.262778  0.269839 -2.530187   56.069697
 2      ovLUW  0.194587  0.289035 -5.586433   62.871910
 3      q6cA6  2.236060 -0.553760  0.930038  114.572842
 4      WPMUX -0.515993  1.716266  5.899011  149.600746
 ...      ...       ...       ...       ...         ...
 99995  4GxBu -1.777037  1.125220  6.263374  172.327046
 99996  YKFjq -1.261523 -0.894828  2.524545  138.748846
 99997  tKPY3 -1.199934 -2.957637  5.219411  157.080080
 99998  nmxp2 -2.419896  2.417221 -5.548444   51.795253
 99999  V9kWn -2.551421 -2.025625  6.090891  102.775767
 
 [100000 rows x 5 columns],
 None)

In [32]:
# splitting and training data
# training set will be 75% source data ; validation set will be 25% source data per project instructions

#region1
region1 = df1.copy()
region1.sort_values('product', ascending=False, inplace=True)
region1.drop_duplicates('id', keep='first', inplace=True)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(region1[['f0', 'f1', 'f2']], region1['product'].values.reshape(-1, 1), test_size=0.25, random_state=123)
model1 = LinearRegression().fit(features_train1, target_train1)    
predictions1 = model1.predict(features_valid1)  
avg_predictions1 = np.mean(predictions1)
region1['predictions'] = model1.predict(region1[['f0', 'f1', 'f2']])
print(f'Average Predicted Reserves for Region 1: {avg_predictions1:.2f}')
rmse = mean_squared_error(target_valid1, predictions1, squared=False)
print(f'RMSE:{rmse:.2f}')

#region2
region2 = df2.copy()
region2.sort_values('product', ascending=False, inplace=True)
region2.drop_duplicates('id', keep='first', inplace=True)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(region2[['f0', 'f1', 'f2']], region2['product'].values.reshape(-1, 1), test_size=0.25, random_state=123)
model2 = LinearRegression().fit(features_train2, target_train2)    
predictions2 = model2.predict(features_valid2)  
avg_predictions2 = np.mean(predictions2)
region2['predictions'] = model2.predict(region2[['f0', 'f1', 'f2']])
print(f'Average Predicted Reserves for Region 2: {avg_predictions2:.2f}')
rmse = mean_squared_error(target_valid2, predictions2, squared=False)
print(f'RMSE:{rmse:.2f}')


#region3
region3 = df3.copy()
region3.sort_values('product', ascending=False, inplace=True)
region3.drop_duplicates('id', keep='first', inplace=True)
features_train3, features_valid3, target_train3, target_valid3 = train_test_split(region3[['f0', 'f1', 'f2']], region3['product'].values.reshape(-1, 1), test_size=0.25, random_state=123)
model3 = LinearRegression().fit(features_train3, target_train3)    
predictions3 = model3.predict(features_valid3)  
avg_predictions3 = np.mean(predictions3)
region3['predictions'] = model3.predict(region3[['f0', 'f1', 'f2']])
print(f'Average Predicted Reserves for Region 3: {avg_predictions3:.2f}')
rmse = mean_squared_error(target_valid3, predictions3, squared=False)
print(f'RMSE:{rmse:.2f}')


Average Predicted Reserves for Region 1: 92.65
RMSE:37.90
Average Predicted Reserves for Region 2: 69.40
RMSE:0.88
Average Predicted Reserves for Region 3: 95.10
RMSE:40.13


Based on this information, Region 3 leads in the predicted average volume with Region 2 following closely behind. They both however have a high margin of error which doesn't necessarily instill the most confidence in their numbers. As for Region 2, it has a lower average of reserves available but also has a very low margin of error, suggesting that the numbers being predicted are more than likely what will be found. 

# Preparing for Profit Calculation

In [23]:
# storing key values for calculations
budget = 100e6  # 100 million USD
revenue_per_unit = 4.5e3  # 4500 USD per thousand barrels
exploration_points = 500
development_points = 200

In [25]:
break_even = (100_000_000 / 200) / 4500
break_even

111.11111111111111

This tells us that there needs to be a minimum volume of 111 thousand per well to break even and create profit.

# Calculating Profit

In [28]:
def calculate_profit(predictions, target, revenue_per_unit, budget, num_wells):
    # Convert to numpy arrays
    predictions = np.array(predictions)
    target = np.array(target)

    # Sort predictions and targets based on highest predicted values
    sorted_indices = predictions.argsort()[-num_wells:]
    selected_predictions = predictions[sorted_indices]
    selected_targets = target[sorted_indices]
    
    # Calculate total predicted volume
    total_volume = selected_targets.sum()
    
    # Calculate potential profit
    revenue = total_volume * revenue_per_unit
    profit = revenue - budget
    
    return profit, total_volume

# region1
profit1, total_volume1 = calculate_profit(predictions1, target_valid1, revenue_per_unit, budget, development_points)
print(f'Region 1 - Total Volume: {total_volume1}, Profit: {profit1}')

# region2
profit2, total_volume2 = calculate_profit(predictions2, target_valid2, revenue_per_unit, budget, development_points)
print(f'Region 2 - Total Volume: {total_volume2}, Profit: {profit2}')

# region3
profit3, total_volume3 = calculate_profit(predictions3, target_valid3, revenue_per_unit, budget, development_points)
print(f'Region 3 - Total Volume: {total_volume3}, Profit: {profit3}')

Region 1 - Total Volume: 10076.339907289777, Profit: -54656470.417196006
Region 2 - Total Volume: 16171.95661892381, Profit: -27226195.214842856
Region 3 - Total Volume: 21258.780355234354, Profit: -4335488.401445404


Based on the calculation above, it's clear that none of the locations meet the requirements to create profit. Each location's top predictions had a total volume that was much less than the minimum to simply break even. Based on this information though, it's also clear that Region 3 has the greatest amount of volume available and will create the least amount of loss. With that, I would recommend Region 3 as the new location simply because it isn't as damaging to the company as the other two regions.

# Calculating Risks and Profit

Bootstrap sampling

In [33]:
def bootstrap(df, n=1000, k=500, top=200):
    profits = []
    
    for _ in range(n):
        # Sample 500 unique wells
        sample = df.sample(k, replace=False)
        
        # Select top 200 wells based on predictions
        top200 = sample.nlargest(top, 'predictions')
        
        # Calculate profit
        profit = top200['product'].sum() * 4500 - 100_000_000
        profits.append(profit)
    
    return np.array(profits)

# Apply for each region
profits1 = bootstrap(region1)
profits2 = bootstrap(region2)
profits3 = bootstrap(region3)

Finding average profit, confidence interval, and risk of losses for each region.

In [36]:
# region1
average_profit1 = np.mean(profits1)
conf_interval1 = np.percentile(profits1, [2.5, 97.5]) # 95% confidence interval
risk_pct1 = (profits1 < 0).mean() * 100

print(f'Average Profit for Region 1: {average_profit1}')
print(f'95% Confidence Interval: {conf_interval1}')
print(f'Risk of Loss for Region 1: {risk_pct1:.2f}%')

# region2
average_profit2 = np.mean(profits2)
conf_interval2 = np.percentile(profits2, [2.5, 97.5])
risk_pct2 = (profits2 < 0).mean() * 100

print(f'Average Profit for Region 2: {average_profit2}')
print(f'95% Confidence Interval: {conf_interval2}')
print(f'Risk of Loss for Region 2: {risk_pct2:.2f}%')

# region3
average_profit3 = np.mean(profits3)
conf_interval3 = np.percentile(profits3, [2.5, 97.5])
risk_pct3 = (profits3 < 0).mean() * 100

print(f'Average Profit for Region 3: {average_profit3}')
print(f'95% Confidence Interval: {conf_interval3}')
print(f'Risk of Loss for Region 3: {risk_pct3:.2f}%')

Average Profit for Region 1: 4246480.195708861
95% Confidence Interval: [-843630.84207598 9005833.19901565]
Risk of Loss for Region 1: 6.30%
Average Profit for Region 2: 4470131.302148249
95% Confidence Interval: [ 408848.55448964 8651803.69286311]
Risk of Loss for Region 2: 1.50%
Average Profit for Region 3: 3578131.7642226564
95% Confidence Interval: [-1812405.8972103   8839562.47099121]
Risk of Loss for Region 3: 9.60%


Upon finding the average profit as well as the risk of loss for each region, we can see that Regions 1 and 2 are closer in terms of profit with Region 3 coming in as the least profitable. However, the risk of loss shows that Regions 1 and 3 both are above the threshold wanted by the company, meaning that they have a margin of error too great to be considered for the new development of wells. Region 2 has an average estimated profit of over $4 million. While the confidence innterval does have a large gap from the lowest and highest amounts, the risk of loss is a small amount of 1.5%. So with all of this information, Region 2 is the best location for OilyGiant based on the estimated profit and risk.

# Conclusion

Beginning this project, I was tasked with assisting the mining company OilyGiant with determining which of three locations would be the best place for the development of new wells. When initially preparing the data, it was fairly simple as the data for each region was intact and had no major changes needed. I then moved on to separating into training and validation sets in order to train the data using a Linear Regression model. The predictions on the average amount of volume that each region contained pointed at Region 3 having the most promise. The RMSE values from this analysis was a bit worrisome though, as it suggested a decent margin for error for Regions 1 and 3, whereas Region 2 seemed much more reliable in its numbers. Keeping that in mind, I then moved on to finding information about profit. It was found that each region needed to contain a minimum of just over 11 thousand volumes of oil per well in order to produce a profit for OilyGiant. After some calculation, it was found that none of those regions met that qualification to create a profit. It did let me know that based on the predictions, Region 3 was estimated at having the least amount of loss. The estimated profit (or lack of it) between the three regions did have enough of a distinction that I could only recommend Region 3 as the new location as the numbers had suggested it was going to be the least damaging financially. So I then moved to do some further analysis on profit and risk to see if this was truly the best decision. To do so, I utilized the bootstrapping technique with 1,000 samples to better find the distribution of profit. I also looked for the 95% confidence interval and risk of loss for each region. The threshold for the risk of loss that the new location needed to meet was 2.5%, and only one region met this standard. With this new information in mind, Region 2 is the only reasonable new location for the new developments of wells. This decision was made with the consideration that OilyGiant will make profit from this new region and that based on the RMSE values and risk of loss percentage, they will have security in their decision to invest in this location.